In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from urllib.parse import urljoin
import re

In [2]:
# THE 200 BEST HORROR MOVIES OF ALL TIME BY Rotten Tomatoes

url_rt = 'https://editorial.rottentomatoes.com/guide/best-horror-movies-of-all-time/'
page_rt = requests.get(url_rt)
page_rt


<Response [200]>

In [3]:
soup = BeautifulSoup(page_rt.content, "html.parser")

In [4]:
# Extract the titles, ratings, and links to the film, director, first cast, second cast and poster

titles = []
tmeter_score = []
director_links = []
movie_links = []
poster_links = []
first_cast_links = []
second_cast_links = []

films = soup.find_all('div', class_="row countdown-item")
website_url = 'https://www.rottentomatoes.com/'

# Include titles splitting and links combining for data cleaning

for film in films:
    tmeter_score.append(film.find('span', class_='tMeterScore').text.strip())
    title = film.find('div', class_='article_movie_title').text.strip().split('\n')[0]
    titles.append(re.sub(r'\s*\(\d{4}\)\s*\d+%$', '', title))
    movie_links.append(film.a['href'])
    director_links.append(urljoin(website_url,film.find('div', class_= 'info director').a['href']))
    poster_links.append(film.find('img', class_= 'article_poster').get('src'))
    first_cast_links.append(urljoin(website_url,film.find('div', class_='info cast').a['href']))
    cast_links = film.find('div', class_='info cast').find_all('a')
    if len(cast_links) >= 2:
     second_cast_links.append(urljoin(website_url,cast_links[1]['href']))
    else:
        second_cast_links.append(None)

    
df_extracted_films = pd.DataFrame({'titles': titles, 
                                   'ratings': tmeter_score, 
                                   'director_links': director_links,
                                   'movie_links': movie_links,
                                   'first_cast_links':first_cast_links,
                                   'second_cast_links':second_cast_links,
                                   'poster_links': poster_links
                                  })
df_extracted_films.head()

,titles,ratings,director_links,movie_links,first_cast_links,second_cast_links,poster_links
0,Alien,98%,https://www.rottentomatoes.com/celebrity/ridle...,https://www.rottentomatoes.com/m/alien,https://www.rottentomatoes.com/celebrity/tom_s...,https://www.rottentomatoes.com/celebrity/sigou...,https://resizing.flixster.com/4uZ_1paESM2WNszw...
1,Let the Right One In,98%,https://www.rottentomatoes.com/celebrity/tomas...,https://www.rottentomatoes.com/m/let_the_right...,https://www.rottentomatoes.com/celebrity/kare_...,https://www.rottentomatoes.com/celebrity/lina-...,https://resizing.flixster.com/oLNICHyrHLPL5plt...
2,Aliens,98%,https://www.rottentomatoes.com/celebrity/james...,https://www.rottentomatoes.com/m/1000617-aliens,https://www.rottentomatoes.com/celebrity/sigou...,https://www.rottentomatoes.com/celebrity/carri...,https://resizing.flixster.com/-5ikyG7jMXnwUBPo...
3,Jaws,97%,https://www.rottentomatoes.com/celebrity/steve...,https://www.rottentomatoes.com/m/jaws,https://www.rottentomatoes.com/celebrity/roy_s...,https://www.rottentomatoes.com/celebrity/rober...,https://resizing.flixster.com/xOAeO8lhaRNEJXvQ...
4,The Silence of the Lambs,95%,https://www.rottentomatoes.com/celebrity/jonat...,https://www.rottentomatoes.com/m/silence_of_th...,https://www.rottentomatoes.com/celebrity/jodie...,https://www.rottentomatoes.com/celebrity/antho...,https://resizing.flixster.com/7-lNwGZYmqtDrxyg...


In [5]:
from IPython.display import Image

print(f'''movie: {df_extracted_films['movie_links'][3]}
first cast: {df_extracted_films['first_cast_links'][3]}
''')

Image(url= df_extracted_films['poster_links'][3])

movie: https://www.rottentomatoes.com/m/jaws
first cast: https://www.rottentomatoes.com/celebrity/roy_scheider



In [6]:
# Convert ratings

df_extracted_films['ratings'] = df_extracted_films['ratings'].str.replace('%', '').astype(float)

In [7]:
# Extract names

df_extracted_films['director'] = df_extracted_films['director_links'].str.split('/').str[-1]
df_extracted_films['first_cast'] = df_extracted_films['first_cast_links'].str.split('/').str[-1]
df_extracted_films['second_cast'] = df_extracted_films['second_cast_links'].str.split('/').str[-1]

In [8]:
df_extracted_films_cleaned=pd.DataFrame(df_extracted_films[['titles','ratings','director','first_cast','second_cast']])
df_extracted_films_cleaned.fillna('Not Available')
df_extracted_films_cleaned.head()

,titles,ratings,director,first_cast,second_cast
0,Alien,98.0,ridley_scott,tom_skerritt,sigourney_weaver
1,Let the Right One In,98.0,tomas-alfredson,kare_hedebrant,lina-leandersson
2,Aliens,98.0,james_cameron,sigourney_weaver,carrie-henn
3,Jaws,97.0,steve_spielberg,roy_scheider,robert_shaw
4,The Silence of the Lambs,95.0,jonathan_demme,jodie_foster,anthony_hopkins
